In [162]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
from keras.layers.core import Dense, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
import collections
import nltk
import re

train = pd.read_csv('~/Downloads/train.csv')
test = pd.read_csv('~/Downloads/test.csv')

print(train.shape)
print(test.shape)

(5668, 2)
(1418, 1)


In [163]:
x_train = train['sentence']
y_train = train['label']
x_test = test['sentence']

nltk.download('punkt')
maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
for sentence in (list(train['sentence']) + list(test['sentence'])):
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    words = nltk.word_tokenize(sentence.lower())
    if len(words) > maxlen:
        maxlen = len(words)
    for word in words:
        word_freqs[word] += 1
    num_recs += 1

print("maxlen: %d, vocab size: %d" % (maxlen, len(word_freqs)))

[nltk_data] Downloading package punkt to /Users/mingxie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


maxlen: 38, vocab size: 2121


In [164]:
MAX_FEATURES = 2200
MAX_SENTENCE_LENGTH = 40

# special words: UNK = -1, PAD = 0
vocab = {"UNK": -1, "PAD": 0}
reverse_vocab = {v:k for k, v in vocab.items()}
for idx, word in enumerate([w[0] for w in word_freqs.most_common(MAX_FEATURES - 1)]):
    vocab[word] = idx + 1
    reverse_vocab[idx + 1] = word

In [165]:
# Convert sentences to token sequences
X = np.empty((x_train.shape[0], ), dtype=list)
for i in range(x_train.shape[0]):
    sentence = x_train[i]
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    words = nltk.word_tokenize(sentence.lower())
    seqs = []
    for word in words:
        seqs.append(vocab.get(word, -1))
    X[i] = seqs

X_train = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [166]:
model = Sequential()
model.add(Embedding(MAX_FEATURES, 128, input_length=MAX_SENTENCE_LENGTH, dropout=0.2))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(X_train, y_train, batch_size=32, epochs=5, shuffle=True, validation_split=0.2)

/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/Users/mingxie/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, dropout=0.2, recurrent_dropout=0.2)`
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 4534 samples, validate on 1134 samples
Epoch 1/5


  32/4534 [..............................] - ETA: 5:11 - loss: 0.6981 - acc: 0.3125

  64/4534 [..............................] - ETA: 2:39 - loss: 0.6939 - acc: 0.4688

  96/4534 [..............................] - ETA: 1:48 - loss: 0.6885 - acc: 0.5729

...

4416/4534 [============================>.] - ETA: 0s - loss: 0.0020 - acc: 0.9995

4448/4534 [============================>.] - ETA: 0s - loss: 0.0020 - acc: 0.9996

4480/4534 [============================>.] - ETA: 0s - loss: 0.0019 - acc: 0.9996

4512/4534 [============================>.] - ETA: 0s - loss: 0.0020 - acc: 0.9996

4534/4534 [==============================] - 9s 2ms/step - loss: 0.0019 - acc: 0.9996 - val_loss: 0.0273 - val_acc: 0.9894


In [167]:
x_test.shape

(1418,)

In [168]:
# Convert sentences to token sequences
X_test = np.empty((x_test.shape[0], ), dtype=list)
for i in range(len(test['sentence'])):
    sentence = str(test['sentence'][i])
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    words = nltk.word_tokenize(sentence.lower())
    seqs = []
    for word in words:
        seqs.append(vocab.get(word, -1))
    X_test[i] = seqs

x_test = sequence.pad_sequences(X_test, maxlen=MAX_SENTENCE_LENGTH)

In [170]:
pred = model.predict(x_test)

In [176]:
submission = pd.read_csv('~/Downloads/sample_submission.csv')
for i in range(pred.shape[0]):
    if pred[i, 0] < 0.5:
        submission.loc[i, 'label'] = 0
    else:
        submission.loc[i, 'label'] = 1

submission.to_csv('~/Downloads/submission4.csv', index=False)  # not store index

